In [1]:
import sys
import os
from openai import OpenAI
from dotenv import load_dotenv
# from src.knowledge_base import TranslationKB

# from src.knowledge_base import update_kb
from src.index_manager import build_or_load_chat_engine, rebuild_index
from src.rag_chat import rag_chat, rag_chat_pipeline
from src.llm_fallback import llm_fallback
from src.prompt_builder import build_prompt

/Users/sarahrmclaughlin/Documents/repos/i_spy_an_agent/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


ImportError: cannot import name 'update_kb' from 'src.knowledge_base' (/Users/sarahrmclaughlin/Documents/repos/i_spy_an_agent/src/knowledge_base.py)

In [ ]:
load_dotenv()

### Make sure API key is working, but not displayed

In [ ]:
api_key = os.getenv("OPENAI_API_KEY")
if api_key:
    print("API Key loaded successfully.")
    print(f"API Key Length: {len(api_key)} characters")
    print(f"API Key: {api_key[:4]}")  
else:
    print("Failed to load Open API Key.")


### Connect to OpenAI

In [ ]:
client = OpenAI(
    base_url="https://api.openai.com/v1",
    api_key=os.getenv("OPENAI_API_KEY"),
)

In [ ]:


def main():
    if len(sys.argv) < 2:
        print("Usage: python src/app.py 'Translate <word>'")
        sys.exit(1)

    query = " ".join(sys.argv[1:])

    # Load chat engine (vector index)
    chat_engine = build_or_load_chat_engine()

    # Run pipeline (RAG → fallback → KB update → rebuild index)
    response = rag_chat_pipeline(query, chat_engine)

    # Print results
    print("\n=== Language RAG Project ===")
    print(f"Query:   {query}")
    print(f"Answer:  {response}")


if __name__ == "__main__":
    main()